In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
csvpath = '../../LHC_mimic/mimic3_1.4/derived/notes.cuis.csv'
# csvpath = '/Z/'

In [3]:
df = pd.read_csv(csvpath, names = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'CATEGORY', 'DESCRIPTION', 'OBSERVATIONS'])

In [4]:
df['OBSERVATIONS'] = [obs.split(' ') for obs in df['OBSERVATIONS'] ]

In [5]:
df

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,OBSERVATIONS
0,2543,106163.0,2114-09-29,2114-09-29T06:14,Nursing/other,Report,"[C0042449, C0032740, C2697811, C0700287, C0392..."
1,26127,161400.0,2147-11-10,2147-11-10T17:28,Nursing/other,Report,"[C0700287, C0018808, C0521346, C3539666, C4049..."
2,15794,111464.0,2199-11-30,2199-11-30T10:48,Nursing/other,Report,"[C0000726, C1642390, C2697811, C0332149, C0036..."
3,70963,110511.0,2107-03-03,2107-03-03T03:58,Radiology,CHEST (PORTABLE AP),"[C0336630, C0086582, C3812382, C0175722, C4049..."
4,91460,122523.0,2149-02-27,2149-02-27T09:13,Radiology,CHEST (PORTABLE AP),"[C0018188, C1963215, C0032326, C0700287, C1962..."
...,...,...,...,...,...,...,...
2077198,80658,117314.0,2111-04-18,2111-04-17T22:21,Nursing,Nursing Progress Note,"[C0021925, C2700258, C0034069, C1800706, C0019..."
2077199,27597,117860.0,2201-03-17,2201-03-17T05:57,Nursing,Nursing Progress Note,"[C0235195, C0344106, C0430404, C3843225, C0280..."
2077200,28194,115597.0,2115-05-13,2115-05-13T04:00,Nursing/other,Report,"[C1883717, C0151636, C0215825, C0042313, C1184..."
2077201,32155,NaN,2113-01-02,2113-01-02T10:41,Radiology,CT SINUS/MANDIBLE/MAXILLOFACIAL W/O CONTRAST,"[C0024957, C0030471, C1305231, C0016658, C2745..."


In [6]:
def add_freqs(lst, freqdict):
    for concept in lst:
        if concept in freqdict:
            freqdict[concept]+=1
        else:
            freqdict[concept]=1

In [7]:
vocab_freqs = {}
df['OBSERVATIONS'].apply(lambda x: add_freqs(x, vocab_freqs))

0          None
1          None
2          None
3          None
4          None
           ... 
2077198    None
2077199    None
2077200    None
2077201    None
2077202    None
Name: OBSERVATIONS, Length: 2077203, dtype: object

In [8]:
vocab = sorted(((value, key) for (key,value) in vocab_freqs.items()), reverse=True)


In [9]:
sorted_ids = {tup[1]:i+1 for i,tup in enumerate(vocab)}

In [10]:
sorted_ids['C0700287']

1

In [11]:
import json
with open('cui_sorted_id.json','w') as jsonfile:
    json.dump(sorted_ids, jsonfile)

In [18]:
len(list(vocab_freqs.keys()))

51893

In [22]:
id_dict = {k:v for v,k in enumerate(list(vocab_freqs.keys())) }

In [24]:
import json
with open('cui_vocab_id.json','w') as jsonfile:
    json.dump(id_dict, jsonfile)

In [2]:
##CREATE SPARSE TENSOR USING CUI VOCAB IDS##
import json
import pandas as pd
# import tensorflow as tf

In [3]:
import os

In [14]:
vocab_path = 'cui_sorted_id.json'
with open(vocab_path, 'r') as vocfile:
    vocab = json.load(vocfile)

In [15]:
list(vocab.keys())[1]

'C2697811'

In [5]:
keylist = list(vocab.keys())
dense_cols = len(keylist)

In [6]:
task = 'in-hospital-mortality'
train_path = './data/just_cui_notes/validated/'+task+'/train/'
embed_path = ''

In [7]:
for filename in os.listdir(train_path)[0:2]:
    if filename == 'listfile.csv':
        continue
    filepath = train_path + filename
    df = pd.read_csv(filepath)[['Hours', 'TEXT']]
#     df['TEXT'] =  [list(set(obs.split(' '))) for obs in df.TEXT]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(vocabulary=[word.lower() for word in keylist], binary=True)

In [69]:
spc = pd.DataFrame.sparse.from_spmatrix(cv.fit_transform(df['TEXT']))

In [71]:
# filter = (spc != 0).any()
# spc.loc[:, filter]
spc.insert(0,'Hours',df.Hours)

In [10]:
nparr = cv.fit_transform(df['TEXT'])

In [11]:
nparr.todense()

matrix([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [12]:
from scipy.sparse import hstack
import numpy as np
spc = hstack((np.array(df['Hours'])[:,None],nparr))

In [13]:
spc = [row for row in spc]

TypeError: 'coo_matrix' object is not subscriptable

In [100]:
import scipy
scipy.sparse.save_npz('test_sparse_matrix.npz', spc)


In [101]:
sparse_matrix = scipy.sparse.load_npz('test_sparse_matrix.npz')

In [102]:
sparse_matrix.todense()

matrix([[ 1.90138889,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [12.50138889,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [23.05138889,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [73.31805556,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [74.21805556,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [82.68472222,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

In [1]:
###TEST TF SPARSE
import tensorflow as tf
import numpy as np

example = tf.SparseTensor(indices=[[0], [1], [2]], values=[3, 6, 9], dense_shape=[3])

vocabulary_size = 10
embedding_size = 1
var = np.array([0.0, 1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0])
embeddings = tf.Variable(var)

embed = tf.nn.embedding_lookup_sparse(embeddings, example, None)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(sess.run(embed)) # prints [  9.  36.  81.]

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[ 9. 36. 81.]


In [2]:
print(example)

SparseTensor(indices=Tensor("SparseTensor/indices:0", shape=(3, 1), dtype=int64), values=Tensor("SparseTensor/values:0", shape=(3,), dtype=int32), dense_shape=Tensor("SparseTensor/dense_shape:0", shape=(1,), dtype=int64))
